In [1]:
#needed drivers
import time
import pickle
import requests
from bs4 import BeautifulSoup
import os 
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
import pickle
import random

In [2]:
# if you need to continue your scraping
#database with all used links
#(link:used)
with open("link_used.pkl", "rb") as f:
        link_used=pickle.load(f)
# all obtained text in html and their links 
# #(link:text)
link_tex= {}
# with open("link_tex_eval.pkl", "rb") as f:
#         link_tex=pickle.load(f)

In [3]:
def get_data(url,link_used,link_tex):
    #settings for the websdriver
    options = webdriver.ChromeOptions() 
    # dowload location
    options.add_argument("--headless") 
    driver = webdriver.Chrome(options=options)
    driver.get(url)
    
    time.sleep(1)
    driver.refresh()
    time.sleep(1)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    
    # step 2 find text
    text=soup.find("div", {"class": "tekst"})
    
    if not text:
        link_used[url]="Y"
        return link_used,link_tex
    else:
        #safes link and their text in to the database
        link_tex[url]=str(text)
        # step 3 find links
        links=text.find_all("a", {"class": "popuplink"})
        for a_tag in links:
            #reference text
            link_text = a_tag.get_text(strip=True)
            # link to other court ruling or law
            # Skip empty or whitespace-only links
            if not link_text or link_text.isspace():
                continue
        
            # step 4 add found links to dict with links
            #make the found link in to a usable link
            trans=transform_url(a_tag['href'])
            #checks if not already in the dict
            if trans not in link_used:
                link_used[trans]="N"

    # step 9 set current link as searched .
    link_used[url]="Y"
    driver.close()
    return link_used,link_tex
    

In [4]:
#transform the unicode link to a usable link
def transform_url(url: str) -> str:
    # Extract the id=... part
    m = re.search(r'id=[^&]+', url)
    if not m:
        raise ValueError("No 'id=' parameter found")

    id_param = m.group(0)  # e.g. 'id=http%3A%2F%2F...BJ6879'

    # Build the new URL
    new_url = (
        f"https://linkeddata.overheid.nl/front/portal/document-viewer?"
        f"{id_param}&callback=&dates=&fields="
    )
    return new_url

In [5]:
#chooses a new link from the database
def new_url(link_used):
    x = "N"
    first_key="empty"
    #check if unused links
    for k, v in link_used.items():
        if v == x:
            first_key = k
            break
    if first_key=="empty":
        link_used,first_key=random_new_url(link_used)
        print(first_key)

        
    return link_used,transform_url(first_key)

In [ ]:
# picks a url to start
link_used,url=new_url(link_used)
#choose this number as a minum set size
saves=10
max_number=10
for z in range(0,max_number):
    for x in range(0,saves):
        link_used,link_tex = get_data(url,link_used,link_tex)
        #
        link_used,url=new_url(link_used)
    print("size is:" ,len(link_used),len(link_tex))
    with open("link_used.pkl", "wb") as f:
        pickle.dump(link_used, f)
    with open("link_tex_eva.pkl", "wb") as b:
        pickle.dump(link_tex, b)


size is: 67 2


In [ ]:
with open("link_tex_eval.pkl", "rb") as b:
        link_tex=pickle.load(b)


In [ ]:

def clean_link_text(t):
    """Return cleaned link text or None if invalid."""
    if t is None:
        return None
    stripped = t.strip()
    if stripped == "" or all(ch in string.punctuation for ch in stripped):
        return None
    return stripped


def extract_link_sentences(html: str,nlp,count,label: str = "REFERENCE"):
    soup = BeautifulSoup(html, "html.parser")
    container = soup.find("div", {"class": "tekst"})
    if not container:
        return []

    text = container.get_text(" ")
    doc = nlp(text)
    sentences = [sent.text for sent in doc.sentences]

    # ---- collect valid link texts ----
    links = []
    for a in soup.find_all("a", {"class": "popuplink"}):
        link_text = clean_link_text(a.get_text(strip=True))
        if link_text:
            links.append(link_text)

    if not links:
        return [],[]
    output_ref = []
    output_not = []

    for sent in sentences:
        raw_entities = []

        # ---- collect all possible matches first ----
        pos=[]
        i=0
        search_pos = 0  
        for link in links:
            # exact matches
            pattern = re.compile(re.escape(link))
            match = pattern.search(sent, search_pos)
            if match:
                raw_entities.append((match.start(), match.end(), label))
                search_pos = match.end()  
                pos.append(i)

            i+=1

        if not raw_entities:
            output_not.append((sent, {"entities": []},count))
            continue

        # ---- enforce ordering rule ----
        raw_entities.sort(key=lambda x: x[0])  # sort by start offset

        filtered_entities = []
        last_start = -1
        k=0
        d=0
        for start, end, lbl in raw_entities:
            if start > last_start:
                filtered_entities.append((start, end, lbl))
                last_start = start
                links.pop(pos[k]-d)
                d+=1
            k+=1
            # else: ignore match (starts before or at same position)

        if filtered_entities:
            output_ref.append((sent, {"entities": filtered_entities},count))
    return output_ref,output_not

In [ ]:
# we only need the html text
texts=link_tex.values()
#all found references
training_data_ref=[]
#all sentences without refrences
training_data_neg=[]
count=0

for text in texts:
    result_ref,result_neg=extract_link_sentences(text,nlp,count)
    if result_ref != []:
        training_data_ref.extend(result_ref)
        training_data_neg.extend(result_neg)
    else:
        print("empty")
    count+=1
    if count%500==0:
        print(count)
    

In [ ]:
examples=training_data_ref+training_data_neg

In [ ]:
    with open("eval_data.pkl", "wb") as b:
        pickle.dump(examples, b)
